In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import pickle

from joblib import delayed, wrap_non_picklable_objects
# sbi
import sbi.utils as utils
from sbi.inference.base import infer

from pathos.pools import ProcessPool

import dill as pickle

# Neuron
from neuron import h

In [3]:
# Neuron Sim
h.load_file("stdrun.hoc")  # for run control
h.load_file("HHNeuron.hoc")  # run the model

def test_parameters(params):
    
    x, y, z = params

    h.initHH()
    # change the values of the model in Neuron
    for c in range(h.HHcell.__len__()):
        h.HHcell[c].isoma.gl_hh = x
        h.HHcell[c].isoma.gkbar_hh = y
        h.HHcell[c].isoma.gnabar_hh = z
    #  Run the model
    h.runHHsim()
    
    n_shape = [h.data_vecs_HH.__len__(), h.data_vecs_HH[0].__len__()]
    
    if n_shape[0] > 0:
        out_voltage = np.empty((0, 3001))
        for y in range(n_shape[0]): # n_shape[0]
            dummy = h.data_vecs_HH[y].as_numpy()[None,]
            out_voltage = np.concatenate((out_voltage, dummy), axis=0)

    Error = calculate_summary_statistics(
        out_voltage
    )
    
    return Error

def calculate_summary_statistics(voltTrace):   
    return torch.tensor(voltTrace[3, 100:102])

In [14]:
#             gl,  gK, gNa
prior_min = [0.00001, 0.001, 0.01]  # fields[0], fields[1], ....
prior_max = [0.001, 0.1, 1.0]  # fields[0], fields[1], ....
num_posterior_draw = 10
num_simulations = 100

prior = utils.torchutils.BoxUniform(low=torch.as_tensor(prior_min), high=torch.as_tensor(prior_max))

In [15]:
from sbi.inference import prepare_for_sbi, SNPE

In [16]:
simulator, prior = prepare_for_sbi(test_parameters, prior)

In [21]:
inference = SNPE(simulator, prior, num_workers=2, mp_framework="mp_pathos")

In [22]:
posterior = inference(num_simulations=num_simulations)


simulation_outputs torch.Size([100, 2])
Neural network successfully converged after 172 epochs.


In [5]:
#infer_obj = infer(test_parameters, prior, method='SNPE', num_simulations=num_simulations, num_workers=2)

PicklingError: Could not pickle the task to send it to the workers.